# iRODS Configuration

This cell should be executed before any other cell in the notebook. It creates the iRODS configuration file and initializes the iRODS connection.

In [ ]:
import getpass
import json
import os
import os.path
import subprocess

def build_irods_config():
    "Builds the iRODS configuration JSON."
    config = {
        "irods_host": "data.cyverse.org",
        "irods_port": 1247,
        "irods_user_name": os.getenv("IPLANT_USER"),
        "irods_zone_name": "iplant",
    }
    return json.dumps(config, indent=4)

def run_shell_cmd(args):
    "Runs a shell command and displays the output."
    proc = subprocess.run(args, capture_output=True)
    if proc.returncode != 0:
        print("\nCommand failed with return code: ", proc.returncode)
    if len(proc.stdout) != 0:
        print("\nStandard Output:\n", proc.stdout.decode())
    if len(proc.stderr) != 0:
        print("\nStandard Error Output:\n", proc.stderr.decode())

# Create the iRODS configuration directory if it doesn't exist.
config_dir = os.path.expanduser("~/.irods")
os.makedirs(config_dir, mode=0o700, exist_ok=True)

# Build the iRODS configuration file if it doesn't already exist.
config_file = os.path.join(config_dir, "irods_environment.json")
if not os.path.exists(config_file):
    with open(config_file, "w") as f:
        print(build_irods_config(), file=f)

# Prompt for the password.
print("Password: ", end='', flush=True)
password = getpass.getpass()

# Initialize the iRODS connection.
run_shell_cmd(["iinit", password])

# Data Synchronization

After executing the previous cell, this cell can be executed as many times as desired in order to copy your output data to the CyVerse data store.

In [ ]:
import os
import os.path
import re

# Determine the source and destination paths.
source_dir = os.path.expanduser("~")
dest_dir = os.path.join("/iplant/home", os.getenv("IPLANT_USER"), "data")

# Make sure that the destination path exists.
run_shell_cmd(["imkdir", "-p", dest_dir])

# Get the list of paths to back up; hidden files and directories are omitted.
paths = [ path for path in os.listdir(source_dir) if not re.match(r"^[.].*", path) ]

# Copy each of the paths back to the data store.
for path in paths:
    source_path = os.path.join(source_dir, path)
    dest_path = os.path.join(dest_dir, path)
    if os.path.isdir(source_path):
        run_shell_cmd(["irsync", "-r", source_path, "i:{0}".format(dest_path)])
    elif os.path.isfile(source_path):
        run_shell_cmd(["irsync", source_path, "i:{0}".format(dest_path)])